In [1]:
import requests
import json
import re
import pandas as pd
import sklearn
import os
import nltk
import string
import newspaper

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from newspaper import Article

In [3]:
########## BUILD CSV WITH LABELS, DESCRIPTION, AND CONTENT ##########

api_key = "4fe8b78ffdb64b0fb8dbfda6a3ba99f6"
url = "https://newsapi.org/v2/everything"

url_paul = ("https://newsapi.org/v2/everything?q=McCartney&domains=rollingstone.com&langauge=en&apiKey=4fe8b78ffdb64b0fb8dbfda6a3ba99f6")
url_john = ("https://newsapi.org/v2/everything?q=Lennon&domains=rollingstone.com&langauge=en&apiKey=4fe8b78ffdb64b0fb8dbfda6a3ba99f6")

params_paul = {"q": "Paul McCartney",
               "language": "en",
               "sortBy": "publishedAt",
               "apiKey": api_key}

params_john = {"q": "John Lennon",
               "language": "en",
               "sortBy": "publishedAt",
               "apiKey": api_key}

response_paul = requests.get(url_paul)
response_john = requests.get(url_john)

if response_paul.status_code == 200:
    json_txt_paul = response_paul.json()
    paul_articles = json_txt_paul.get("articles", [])
    df_paul = pd.DataFrame(paul_articles)

else:
    print(f"Error: {response_paul.status_code}, {response_paul.text}")

df_paul['full_text'] = df_paul['url'].apply(lambda url: (lambda a: (a.download(), a.parse(), a.text)[2])(Article(url)))
df_paul['label'] = len(df_paul)*['paul']

if response_john.status_code == 200:
    json_txt_john = response_john.json()
    john_articles = json_txt_john.get("articles", [])
    df_john = pd.DataFrame(john_articles)

else:
    print(f"Error: {response_john.status_code}, {response_john.text}")

df_john['full_text'] = df_john['url'].apply(lambda url: (lambda a: (a.download(), a.parse(), a.text)[2])(Article(url)))
df_john['label'] = len(df_john)*['john']

df_paul_john = pd.concat([df_paul, df_john])
df_paul_john_description = df_paul_john[['label', 'description']]
df_paul_john_full_text = df_paul_john[['label', 'full_text']]
df_paul_john_full_text.loc[:, 'full_text'] = df_paul_john_full_text['full_text'].replace(r'\n', ' ', regex=True)

df_paul_john_description.to_csv("paul_john_description.csv", index = True)
df_paul_john_full_text.to_csv("paul_john_full_text.csv", index = True)

In [4]:
########## USE CSV TO BUILD DATAFRAME ##########
# First, for descriptions

# Path name
path = "C:/Users/maede/Downloads/Python/INFO5653/Project/Project_Part1/INFO5653_Project/paul_john_description.csv"

# Open path, create file, open file, write column names, close file
file = open(path,"r", encoding="utf-8")
filename = "paul_john_description_clean.csv"
new_file = open(filename,"w", encoding="utf-8")
to_write = "Label,Description\n"
new_file.write(to_write)
new_file.close()

# Open file to append to
new_file = open(filename, "a", encoding="utf-8")

# Initliaze empty dataframe
paul_john_descrip_clean_df = pd.DataFrame()
output_file = "paul_john_description_clean.txt"
outfile = open(output_file,"w", encoding="utf-8")
outfile.close()
outfile = open(output_file,"a", encoding="utf-8") 

# Skip reading in the column names
next(file) 

for row in file:
    # Remove any necessary spaces from each row
    row = row.lstrip()
    row = row.rstrip() 
    row = row.strip()

    # Split at each space to look at words
    words = re.split(" ", row)
    # print(words)
    words_clean = []

    for word in words:
        # Clean words!
        word = word.lower()
        word = word.lstrip()
        # word = word.replace(",","") 
        word = word.replace(" ","")
        word = word.replace("_","" )
        word = re.sub('\+', ' ', word) 
        word = re.sub('.*\+\n', '', word)
        word = re.sub('zz+', ' ', word)
        word = word.replace("\t","")
        word = word.replace(".","")
        word = word.strip()
        words_clean.append(word)

    label = words_clean[0].split(",")[1]
    first_word = words_clean[0].split(",")[2]
    words_clean[0] = first_word
    
    if "john" in label:
        label = "john"
    else:
        label = "paul"
        
    text = " ".join(words_clean)
    
    ### More cleaning....
    text = text.replace("\\n","")
    text = text.strip("\\n")
    text = text.replace("\\'","")
    text = text.replace("\\","")
    text = text.replace('"',"")
    text = text.replace("'","")
    text = text.replace(",","")
    text = text.replace("s'","")
    text = text.lstrip()

    # Write to files
    write = label + ","+ text + "\n"
    new_file.write(write)
    outfile.write(write)
     
# Close files
file.close()  
new_file.close()
outfile.close()

# Read in clean csv to use count vectorizor on
clean = pd.read_csv(filename)

# Some cleaning
clean = clean.dropna(how = 'any', axis = 0)  ## axis 0 is rowwise

# Store labels
labels = clean["Label"]

# Store data without labels
data = clean.drop(["Label"], axis=1) 

# Build a list of content that countvectorizer expects.
my_list = [] 

for i in range(0, len(data)):
    next_text = data.iloc[i,0]
    my_list.append(next_text)

# Now vectorize
my_vect = CountVectorizer(input = 'content', stop_words = 'english', token_pattern=r'\b[a-zA-Z]{4,}\b', max_features = 50)

fit = my_vect.fit_transform(my_list)

# Add columns names and labels
column_names = my_vect.get_feature_names_out()
paul_john_descrip_clean_df = pd.DataFrame(fit.toarray(), columns = column_names)
paul_john_descrip_clean_df = paul_john_descrip_clean_df.rename(labels, axis = "index")
paul_john_descrip_clean_df.index.name = "Label"

# Write dataframe to csv
paul_john_descrip_clean_df.to_csv("paul_john_description_clean.csv", index = True)

In [7]:
########## USE CSV TO BUILD DATAFRAME ##########
# Now for content

# Path name
path2 = "C:/Users/maede/Downloads/Python/INFO5653/Project/Project_Part1/INFO5653_Project/paul_john_full_text.csv"

# Open path, create file, open file, write column names, close file
file2 = open(path2,"r", encoding="utf-8")
filename2 = "paul_john_full_text_clean.csv"
new_file2 = open(filename2,"w", encoding="utf-8")
to_write2 = "Label,Content\n"
new_file2.write(to_write2)
new_file2.close()

# Open file to append to
new_file2 = open(filename2, "a", encoding="utf-8")

# Initliaze empty dataframe
paul_john_content_clean_df = pd.DataFrame()
output_file2 = "paul_john_full_text_clean.txt"
outfile2 = open(output_file2,"w", encoding="utf-8")
outfile2.close()
outfile2 = open(output_file2,"a", encoding="utf-8") 

# Skip reading in the column names
next(file2) 
for row in file2:
    # Remove any necessary spaces from each row
    row = row.lstrip()
    row = row.rstrip() 
    row = row.strip()
    row = re.sub(r'\n', " ", row)

    # Split at each space to look at words
    words = re.split(" ", row)
    # print(words)
    words_clean = []

    for word in words:
        # Clean words!
        word = word.lower()
        word = word.lstrip()
        # word = word.replace(",","") 
        word = word.replace(" ","")
        word = word.replace("_","" )
        word = re.sub('\+', ' ', word) 
        word = re.sub('.*\+\n', '', word)
        word = re.sub('zz+', ' ', word)
        word = word.replace("\t","")
        word = word.replace(".","")
        word = word.strip()
        words_clean.append(word)

    label = words_clean[0].split(",")[1]
    first_word = words_clean[0].split(",")[2]
    words_clean[0] = first_word
    
    if "john" in label:
        label = "john"
    else:
        label = "paul"
        
    text = " ".join(words_clean)
    
    ### More cleaning....
    text = text.replace("\\n","")
    text = text.strip("\\n")
    text = text.replace("\\'","")
    text = text.replace("\\","")
    text = text.replace('"',"")
    text = text.replace("'","")
    text = text.replace(",","")
    text = text.replace("s'","")
    text = text.lstrip()

    # Write to files
    write = label + ","+ text + "\n"
    new_file2.write(write)
    outfile2.write(write)
     
# Close files
file2.close()  
new_file2.close()
outfile2.close()

# Read in clean csv to use count vectorizor on
clean2 = pd.read_csv(filename2)

# Some cleaning
clean2 = clean2.dropna(how = 'any', axis = 0)  ## axis 0 is rowwise

# Store labels
labels2 = clean2["Label"]

# Store data without labels
data2 = clean2.drop(["Label"], axis=1) 

# Build a list of content that countvectorizer expects.
my_list2 = [] 

for i in range(0, len(data2)):
    next_text2 = data2.iloc[i,0]
    my_list2.append(next_text2)

# Now vectorize
my_vect2 = CountVectorizer(input = 'content', stop_words = 'english', token_pattern=r'\b[a-zA-Z]{4,}\b', max_features = 100)

fit2 = my_vect2.fit_transform(my_list2)

# Add columns names and labels
column_names2 = my_vect2.get_feature_names_out()
paul_john_full_text_clean_df = pd.DataFrame(fit2.toarray(), columns = column_names2)
paul_john_full_text_clean_df = paul_john_full_text_clean_df.rename(labels2, axis = "index")
paul_john_full_text_clean_df.index.name = "Label"

# Write dataframe to csv
paul_john_full_text_clean_df.to_csv("paul_john_full_text_clean.csv", index = True)